<a href="https://colab.research.google.com/github/Korsholm22/M4_Group_Assignments/blob/main/Group_Assignment_2/Group_Assignment_2_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**TASK**
- Build, train, and evaluate 2 special types of networks neural network with Pytorch.
  - **A CNN for a spatial prediction problem**
  - A RNN or LSTM for a sequential problem
- Experiment with at least 2 different variations of hyperparameters for each network.
- Optional: Use gradio to build a simple interactive demo (in the notebook).

This should include:

1. Feature selection and/or engineering (if necessary)
2. Preprocessing (if necessary)
3. Train-test split
4. Defining a neural network architecture in putorch
5. Define a training loop
6. Training the model
7. Try out different hyperparameters
8. Evaluate the final model on the test data
9. Optional: Visualize results with Grad.io

#CNN
## Importing Library

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torchvision.datasets as datasets
from PIL import Image

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
#Defining Hyper-parameters
num_epochs = 4
batch_size = 4
learning_rate = 0.003

In [3]:
#Transforming images to tensors, 0,5 & 0,5 because the images are greyscale, therefore they dont use RGB colous
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))])

In [4]:
#Importing the datasets which are already split into training and testing
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [5]:
#Automating batch sizes and shuffle the data and labels in the training set
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle=False)

classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

In [6]:
# implement Convolutional networks
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        #feature learning part
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.pool = nn.MaxPool2d(2,2) #kernel_size:2 stride: 2
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        #Classification 
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120) #12*4*4= number of output channels * dimension of the output layer after feature learning; 120 manually set as outcome of the 
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.fc3 = nn.Linear(in_features=60, out_features=10) #the output dimension (in our case 10) needs to be equal to the number of classes
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 12*4*4) #torch.view() function freshape the input tensor
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = ConvNet().to(device)

In [7]:
criterion = nn.CrossEntropyLoss() #CrossEntropyLoss already includes SoftMax
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) # optimizing the model with gradient decent


In [8]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 1 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad() #with zero_grad() we ensure that the gradients are properly reset to zero at the start of each iteration
        loss.backward()
        optimizer.step()


        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
            model.eval()
            with torch.no_grad():
                correct = 0
                total = 0
                for images, labels in test_loader:
                    images = images.to(device)
                    labels = labels.to(device)
                    outputs = model(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                accuracy = correct / total
        
print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

Epoch [1/4], Step [2000/15000], Loss: 1.2172
Epoch [1/4], Step [4000/15000], Loss: 0.6100
Epoch [1/4], Step [6000/15000], Loss: 0.0214
Epoch [1/4], Step [8000/15000], Loss: 0.0086
Epoch [1/4], Step [10000/15000], Loss: 0.0210
Epoch [1/4], Step [12000/15000], Loss: 0.0149
Epoch [1/4], Step [14000/15000], Loss: 0.0971
Epoch [2/4], Step [2000/15000], Loss: 0.0015
Epoch [2/4], Step [4000/15000], Loss: 0.0082
Epoch [2/4], Step [6000/15000], Loss: 0.0018
Epoch [2/4], Step [8000/15000], Loss: 0.0456
Epoch [2/4], Step [10000/15000], Loss: 0.0063
Epoch [2/4], Step [12000/15000], Loss: 0.0003
Epoch [2/4], Step [14000/15000], Loss: 0.0091
Epoch [3/4], Step [2000/15000], Loss: 0.0004
Epoch [3/4], Step [4000/15000], Loss: 0.0002
Epoch [3/4], Step [6000/15000], Loss: 0.0261
Epoch [3/4], Step [8000/15000], Loss: 0.0032
Epoch [3/4], Step [10000/15000], Loss: 0.0188
Epoch [3/4], Step [12000/15000], Loss: 0.0084
Epoch [3/4], Step [14000/15000], Loss: 0.0018
Epoch [4/4], Step [2000/15000], Loss: 0.0006
E

In [23]:
# In this step we test the accuracy of the model while also seeing how well it does with the individual numbers 
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Accuracy of the network: 99.06 %
Accuracy of 0: 99.59183673469387 %
Accuracy of 1: 99.55947136563877 %
Accuracy of 2: 99.03100775193798 %
Accuracy of 3: 99.00990099009901 %
Accuracy of 4: 99.08350305498982 %
Accuracy of 5: 98.87892376681614 %
Accuracy of 6: 98.8517745302714 %
Accuracy of 7: 98.83268482490273 %
Accuracy of 8: 99.07597535934292 %
Accuracy of 9: 98.61248761149653 %


### Hyperparameter changes

#### 1st changes - Learning rate

In [14]:
num_epochs = 4
batch_size = 4
learning_rate_2 = 0.0005

In [15]:
optimizer_2 = torch.optim.SGD(model.parameters(), lr=learning_rate_2)

In [16]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        #images = images.reshape(-1, 28*28).to(device)
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer_2.zero_grad() #with zero_grad() we ensure that the gradients are properly reset to zero at the start of each iteration
        loss.backward() 
        optimizer_2.step()
        
        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
            # Calculate average accuracy for every 2000 steps
            model.eval()
            with torch.no_grad():
                correct = 0
                total = 0
                for images, labels in test_loader:
                    images = images.to(device)
                    labels = labels.to(device)
                    outputs = model(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                accuracy = correct / total
                
print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

Epoch [1/4], Step [2000/15000], Loss: 0.0006
Epoch [1/4], Step [4000/15000], Loss: 0.0012
Epoch [1/4], Step [6000/15000], Loss: 0.0060
Epoch [1/4], Step [8000/15000], Loss: 1.3580
Epoch [1/4], Step [10000/15000], Loss: 0.0005
Epoch [1/4], Step [12000/15000], Loss: 0.0003
Epoch [1/4], Step [14000/15000], Loss: 0.0011
Epoch [2/4], Step [2000/15000], Loss: 0.0002
Epoch [2/4], Step [4000/15000], Loss: 0.0003
Epoch [2/4], Step [6000/15000], Loss: 0.0010
Epoch [2/4], Step [8000/15000], Loss: 0.0002
Epoch [2/4], Step [10000/15000], Loss: 0.0005
Epoch [2/4], Step [12000/15000], Loss: 0.0003
Epoch [2/4], Step [14000/15000], Loss: 0.0246
Epoch [3/4], Step [2000/15000], Loss: 0.0007
Epoch [3/4], Step [4000/15000], Loss: 0.0042
Epoch [3/4], Step [6000/15000], Loss: 0.0050
Epoch [3/4], Step [8000/15000], Loss: 0.0035
Epoch [3/4], Step [10000/15000], Loss: 0.0009
Epoch [3/4], Step [12000/15000], Loss: 0.0001
Epoch [3/4], Step [14000/15000], Loss: 0.0116
Epoch [4/4], Step [2000/15000], Loss: 0.0007
E

In [22]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size_2):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')
     

Accuracy of the network: 99.06 %
Accuracy of 0: 99.59183673469387 %
Accuracy of 1: 99.55947136563877 %
Accuracy of 2: 99.03100775193798 %
Accuracy of 3: 99.00990099009901 %
Accuracy of 4: 99.08350305498982 %
Accuracy of 5: 98.87892376681614 %
Accuracy of 6: 98.8517745302714 %
Accuracy of 7: 98.83268482490273 %
Accuracy of 8: 99.07597535934292 %
Accuracy of 9: 98.61248761149653 %


#### 2nd Changes - Epocs

In [18]:
num_epochs_2 = 50

In [19]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs_2):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        #images = images.reshape(-1, 28*28).to(device)
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad() #with zero_grad() we ensure that the gradients are properly reset to zero at the start of each iteration
        loss.backward() 
        optimizer.step()
        
        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs_2}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
            # Calculate average accuracy for every 2000 steps
            model.eval()
            with torch.no_grad():
                correct = 0
                total = 0
                for images, labels in test_loader:
                    images = images.to(device)
                    labels = labels.to(device)
                    outputs = model(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                accuracy = correct / total
                
print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

Epoch [1/50], Step [2000/15000], Loss: 0.0424
Epoch [1/50], Step [4000/15000], Loss: 0.0048
Epoch [1/50], Step [6000/15000], Loss: 0.0019
Epoch [1/50], Step [8000/15000], Loss: 0.0573
Epoch [1/50], Step [10000/15000], Loss: 0.0004
Epoch [1/50], Step [12000/15000], Loss: 0.1346
Epoch [1/50], Step [14000/15000], Loss: 0.0614
Epoch [2/50], Step [2000/15000], Loss: 0.0326
Epoch [2/50], Step [4000/15000], Loss: 0.0014
Epoch [2/50], Step [6000/15000], Loss: 0.0022
Epoch [2/50], Step [8000/15000], Loss: 0.1049
Epoch [2/50], Step [10000/15000], Loss: 0.0002
Epoch [2/50], Step [12000/15000], Loss: 0.0011
Epoch [2/50], Step [14000/15000], Loss: 0.0003
Epoch [3/50], Step [2000/15000], Loss: 0.0013
Epoch [3/50], Step [4000/15000], Loss: 0.0047
Epoch [3/50], Step [6000/15000], Loss: 0.0003
Epoch [3/50], Step [8000/15000], Loss: 0.0003
Epoch [3/50], Step [10000/15000], Loss: 0.0013
Epoch [3/50], Step [12000/15000], Loss: 0.0002
Epoch [3/50], Step [14000/15000], Loss: 0.0010
Epoch [4/50], Step [2000/

In [21]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size_2):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Accuracy of the network: 99.06 %
Accuracy of 0: 99.59183673469387 %
Accuracy of 1: 99.55947136563877 %
Accuracy of 2: 99.03100775193798 %
Accuracy of 3: 99.00990099009901 %
Accuracy of 4: 99.08350305498982 %
Accuracy of 5: 98.87892376681614 %
Accuracy of 6: 98.8517745302714 %
Accuracy of 7: 98.83268482490273 %
Accuracy of 8: 99.07597535934292 %
Accuracy of 9: 98.61248761149653 %
